In [1]:
# import necessary libraries
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.model_selection import train_test_split
import time

In [2]:
#dictionary with usernames their watched web series and ratings out of 5

dataset={
        'Tanya': {'Special Ops': 5,
                  'Criminal Justice': 3,
                  'Panchayat': 3,
                  'Sacred Games': 3,
                  'Apharan': 2,
                  'Mirzapur': 3},
    
        'Mohan': {'Special Ops': 5,
                    'Criminal Justice': 3,
                    'Sacred Games': 5,
                    'Panchayat':5,
                    'Mirzapur': 3,
                    'Apharan': 3},
    
        'Tasha': {'Special Ops': 2,
                   'Panchayat': 5,
                   'Sacred Games': 3,
                   'Mirzapur': 4},
    
        'Nirbhay': {'Panchayat': 5,
                   'Mirzapur': 4,
                   'Sacred Games': 4,},
    
       'Muskan': {'Special Ops': 4,
                    'Criminal Justice': 4,
                    'Panchayat': 4,
                    'Mirzapur': 3,
                    'Apharan': 2},
    
       'Anshika': {'Special Ops': 3,
                  'Panchayat': 4,
                  'Mirzapur': 3,
                  'Sacred Games': 5},
    
      'Dhawal': {'Panchayat':4,
                  'Apharan':1,
                  'Sacred Games':4}}

In [3]:
#create a data frame of this dataset

dataset_df=pd.DataFrame(dataset)
dataset_df.fillna("Not Seen Yet",inplace=True)
dataset_df

,Tanya,Mohan,Tasha,Nirbhay,Muskan,Anshika,Dhawal
Special Ops,5,5,2.0,Not Seen Yet,4.0,3.0,Not Seen Yet
Criminal Justice,3,3,Not Seen Yet,Not Seen Yet,4.0,Not Seen Yet,Not Seen Yet
Panchayat,3,5,5.0,5.0,4.0,4.0,4.0
Sacred Games,3,5,3.0,4.0,Not Seen Yet,5.0,4.0
Apharan,2,3,Not Seen Yet,Not Seen Yet,2.0,Not Seen Yet,1.0
Mirzapur,3,3,4.0,4.0,3.0,3.0,Not Seen Yet


In [4]:
# custom function to create cosine correlation method from scratch

def person_corelation(person1,person2):
    both_rated = {}
    for item in dataset[person1]:
        if item in dataset[person2]:
            both_rated[item] = 1

    number_of_ratings = len(both_rated)
    if number_of_ratings == 0:
        return 0
    # Mean of each user
    person1_mean = sum(dataset[person1][item] for item in dataset[person1])/len(dataset[person1])
    person2_mean = sum(dataset[person2][item] for item in dataset[person2])/len(dataset[person2])
    
    # Sum up the squares of preferences of each user
    person1_square_preferences_sum = sum([pow(dataset[person1][item]-person1_mean, 2) for item in both_rated])
    person2_square_preferences_sum = sum([pow(dataset[person2][item]-person2_mean, 2) for item in both_rated])

    # Sum up the product value of both preferences for each item
    product_sum_of_both_users = sum([(dataset[person1][item]-person1_mean) * (dataset[person2][item]-person2_mean) for item in both_rated])

    # Calculate the pearson score
    denominator_value = sqrt(person1_square_preferences_sum)*sqrt(person2_square_preferences_sum)
    if denominator_value == 0:
        return 0
    else:
        r = product_sum_of_both_users / denominator_value
        return r

In [5]:
print(person_corelation('Tanya','Anshika'))

-0.487377324966494


In [6]:
print(person_corelation('Mohan','Anshika'))

0.45226701686664544


In [7]:
print(person_corelation('Muskan','Anshika'))

-1.0860556632786142e-16


In [8]:
print(person_corelation('Dhawal','Anshika'))

0.8320502943378437


In [9]:
def find_users_watched_movie(movie):
    users_watched = []
    for user, movies in dataset.items():
        if movie in movies:
            users_watched.append(user)
    return users_watched

In [10]:
# custom function to check most similar users

def most_similar_users(target_person,movie,no_of_users):
    users_have_seen_movie = find_users_watched_movie(movie)
    
    # Used list comprehension for finding pearson similarity between users
    scores = [(person_corelation(target_person,other_person),other_person) for other_person in dataset if other_person != target_person and other_person in users_have_seen_movie]
    
    #sort the scores in descending order
    scores.sort(reverse=True)
    if no_of_users < len(users_have_seen_movie):
        #return the scores between the target person & other persons
        k = no_of_users
        similar_users = [[score,user] for score, user in scores[0:no_of_users]]
    else:
        #return the scores between the target person & other persons
        k = len(users_have_seen_movie)
        similar_users = [[score,user] for score, user in scores[0:len(users_have_seen_movie)]]
    return [similar_users,k]

In [11]:
def predict_each_movie(person,movie,k):
    
    if person in dataset.keys():
        similar_person = most_similar_users(person,movie,k)
        sum_rating = 0
        for user in similar_person[0]:
            sum_rating = sum_rating + dataset[user[1]][movie]
    return sum_rating/similar_person[1]

In [16]:
print(predict_each_movie('Anshika','Apharan',3))

2.0


In [13]:
# from sklearn.metrics.pairwise import cosine_similarity

In [14]:
# array_vec_1 = np.array([[0.25,1.25]])
# array_vec_2 = np.array([[1,1]]) 

In [15]:
# print(cosine_similarity(array_vec_1,array_vec_2))